<a href="https://colab.research.google.com/github/nirob271/learning-ml-dl-fl/blob/main/Diabates_Data_Preprocessing_and_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


# **Loading and Exploring the Dataset**

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/diabetes.csv')

print("Dataset Shape:", df.shape)

print("\nFirst 5 rows:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nBasic Statistics:")
print(df.describe())

print("\nCheck for missing values:")
print(df.isnull().sum())


print("\nTarget variable distribution:")
print(df['Outcome'].value_counts())


Dataset Shape: (768, 9)

First 5 rows:
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 

# **Train-Test Split**

In [4]:
# Separate input (X) and output (y)
X = df.drop('Outcome',axis=1) # Everything except outcome
y = df['Outcome'] # Only Outcome

# Split: 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(
    X,y,
    test_size=0.2, # 20% for test
    random_state=43, # same result everytime
)
print(f"\nWe will train with: {len(X_train)} patients")
print(f"We will test with: {len(X_test)} patients")



We will train with: 614 patients
We will test with: 154 patients


# **StandardScaler**

In [5]:
# Create scaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data
X_test_scaled = scaler.transform(X_test)


print("\nAfter scaling:")
print("All features now have similar range!")
print(f"Example - First patient's age before scaling: {X_train.values[0][7]}")
print(f"Example - First patient's age after scaling: {X_train_scaled[0][7]:.2f}")


After scaling:
All features now have similar range!
Example - First patient's age before scaling: 25.0
Example - First patient's age after scaling: -0.70


# **MinMaxScaler (Min-Max normalization)**

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Create MinMaxScaler
minmax_scaler = MinMaxScaler()

# Learn from training data and scale it
X_train_minmax = minmax_scaler.fit_transform(X_train)

# Scale test data
X_test_minmax = minmax_scaler.transform(X_test)

print("\nAfter MinMaxScaler:")
print("All features now between 0 and 1!")
print(f"Example - First patient's age before scaling: {X_train.values[0][7]}")
print(f"Example - First patient's age after scaling: {X_train_minmax[0][7]:.2f}")


After MinMaxScaler:
All features now between 0 and 1!
Example - First patient's age before scaling: 25.0
Example - First patient's age after scaling: 0.07


# **Encoding Categorical Data**

In [7]:
# Let's create a sample dataset with text data
sample_data = pd.DataFrame({
    'age': [25, 30, 35, 40],
    'gender': ['male', 'female', 'male', 'female'],
    'smoker': ['yes', 'no', 'yes', 'no'],
    'disease': [1, 0, 1, 0]
})

print("Original data with text:")
print(sample_data)

Original data with text:
   age  gender smoker  disease
0   25    male    yes        1
1   30  female     no        0
2   35    male    yes        1
3   40  female     no        0


# **Label Encoding **

In [8]:
from sklearn.preprocessing import LabelEncoder

# For yes/no or male/female (2 categories only)
label_encoder = LabelEncoder()

# Convert 'smoker' column
sample_data['smoker_encoded'] = label_encoder.fit_transform(sample_data['smoker'])
print("\nAfter Label Encoding 'smoker':")
print(sample_data[['smoker', 'smoker_encoded']])
# no = 0, yes = 1

# Convert 'gender' column
sample_data['gender_encoded'] = label_encoder.fit_transform(sample_data['gender'])
print("\nAfter Label Encoding 'gender':")
print(sample_data[['gender', 'gender_encoded']])
# female = 0, male = 1


After Label Encoding 'smoker':
  smoker  smoker_encoded
0    yes               1
1     no               0
2    yes               1
3     no               0

After Label Encoding 'gender':
   gender  gender_encoded
0    male               1
1  female               0
2    male               1
3  female               0


# **One-Hot Encoding**

In [9]:
# Let's add a column with multiple categories
sample_data['city'] = ['Delhi', 'Mumbai', 'Delhi', 'Kolkata']

# One-Hot Encoding using pandas
city_encoded = pd.get_dummies(sample_data['city'], prefix='city')
print("\nOne-Hot Encoding for 'city':")
print(city_encoded)
# Creates separate columns for each city

# Add to original data
sample_data = pd.concat([sample_data, city_encoded], axis=1)
print("\nFinal data with all encodings:")
print(sample_data)


One-Hot Encoding for 'city':
   city_Delhi  city_Kolkata  city_Mumbai
0        True         False        False
1       False         False         True
2        True         False        False
3       False          True        False

Final data with all encodings:
   age  gender smoker  disease  smoker_encoded  gender_encoded     city  \
0   25    male    yes        1               1               1    Delhi   
1   30  female     no        0               0               0   Mumbai   
2   35    male    yes        1               1               1    Delhi   
3   40  female     no        0               0               0  Kolkata   

   city_Delhi  city_Kolkata  city_Mumbai  
0        True         False        False  
1       False         False         True  
2        True         False        False  
3       False          True        False  


# **Train a Simple Model**

In [10]:
from sklearn.linear_model import LogisticRegression

# Create model
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(X_train_scaled, y_train)
print("✅ Model trained!")

# Check accuracy on training data
train_score = model.score(X_train_scaled, y_train)
print(f"Training accuracy: {train_score:.1%}")

# Check accuracy on test data
test_score = model.score(X_test_scaled, y_test)
print(f"Test accuracy: {test_score:.1%}")

✅ Model trained!
Training accuracy: 78.7%
Test accuracy: 77.9%


# **Cross-Validation**

In [11]:
from sklearn.model_selection import cross_val_score

# Do 5-fold cross-validation
scores = cross_val_score(
    model,           # Our model
    X_train_scaled,  # Training data
    y_train,         # Training labels
    cv=5            # 5 mini-tests
)

print("5 mini-test scores:")
for i in range(5):
    print(f"  Test {i+1}: {scores[i]:.1%}")

print(f"\nAverage: {scores.mean():.1%}")
print(f"This means our model is {scores.mean():.1%} accurate!")

5 mini-test scores:
  Test 1: 78.9%
  Test 2: 83.7%
  Test 3: 75.6%
  Test 4: 74.0%
  Test 5: 76.2%

Average: 77.7%
This means our model is 77.7% accurate!


# **Save Everything**

In [12]:
import joblib

# Save the model
joblib.dump(model, 'my_model.pkl')
print("✅ Model saved as 'my_model.pkl'")

# Save the scaler
joblib.dump(scaler, 'my_scaler.pkl')
print("✅ Scaler saved as 'my_scaler.pkl'")

# Save scaled data as CSV
pd.DataFrame(X_train_scaled).to_csv('train_data_scaled.csv', index=False)
print("✅ Scaled data saved as 'train_data_scaled.csv'")

✅ Model saved as 'my_model.pkl'
✅ Scaler saved as 'my_scaler.pkl'
✅ Scaled data saved as 'train_data_scaled.csv'


# **Use Saved Model for New Patient**

In [23]:
# Load saved model and scaler
model = joblib.load('my_model.pkl')
scaler = joblib.load('my_scaler.pkl')

# New patient data (8 features)
print("\nNew patient information:")
new_patient = [[5, 16, 100, 100, 300, 25.6, 0.201, 70]]
print("Pregnancies: 5")
print("Glucose: 116")
print("Blood Pressure: 74")
print("Age: 30")
print("(and other features...)")

# Scale the new patient data
new_patient_scaled = scaler.transform(new_patient)

# Make prediction
prediction = model.predict(new_patient_scaled)

if prediction[0] == 1:
    print("\n⚠️ Result: Risk of diabetes")
else:
    print("\n✅ Result: No diabetes risk")


New patient information:
Pregnancies: 5
Glucose: 116
Blood Pressure: 74
Age: 30
(and other features...)

✅ Result: No diabetes risk
